
# SIFT feature detector and descriptor extractor

This example demonstrates the SIFT feature detection and its description
algorithm.

The scale-invariant feature transform (SIFT) [1]_ was published in 1999 and is
still one of the most popular feature detectors available, as its promises to
be "invariant to image scaling, translation, and rotation, and partially
in-variant to illumination changes and affine or 3D projection" [2]_. Its
biggest drawback is its runtime, that's said to be "at two orders of
magnitude" [3]_ slower than ORB, which makes it unsuitable for real-time
applications.

## References
.. [1] https://en.wikipedia.org/wiki/Scale-invariant_feature_transform

.. [2] D.G. Lowe. "Object recognition from local scale-invariant
       features", Proceedings of the Seventh IEEE International
       Conference on Computer Vision, 1999, vol.2, pp. 1150-1157.
       :DOI:`10.1109/ICCV.1999.790410`

.. [3] Ethan Rublee, Vincent Rabaud, Kurt Konolige and Gary Bradski
       "ORB: An efficient alternative to SIFT and SURF"
       http://www.gwylab.com/download/ORB_2012.pdf


In [ ]:
import numpy as np
import cv2
from google.colab import drive
import matplotlib.pyplot as plt

# Mount Google Drive
drive.mount('/content/drive')

# Load the two images
img1 = cv2.imread('/content/drive/My Drive/ADVIDEO/panoram1.jpg')
img2 = cv2.imread('/content/drive/My Drive/ADVIDEO/panoram2.jpg')

# Check if the images are loaded correctly
if img1 is None or img2 is None:
    print("Error loading images")
    exit()

# Convert images to grayscale
gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and descriptors
keypoints1, descriptors1 = sift.detectAndCompute(gray1, None)
keypoints2, descriptors2 = sift.detectAndCompute(gray2, None)

# Use BFMatcher to match descriptors
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
matches = bf.match(descriptors1, descriptors2)

# Sort matches based on distance
matches = sorted(matches, key=lambda x: x.distance)

# Draw matches on a new image
matched_image = cv2.drawMatches(img1, keypoints1, img2, keypoints2, matches[:20], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

# Compute homography
src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
H, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

# Warp the first image to align it with the second
height, width, channels = img2.shape
img1_warped = cv2.warpPerspective(img1, H, (width + img1.shape[1], height))

# Place the second image in the correct position on the canvas
img1_warped[0:height, 0:width] = img2

# Show the original images, the matched features, and the resulting panoramic image
plt.figure(figsize=(20, 10))

# Show the first image in the first subplot
plt.subplot(2, 2, 1)
plt.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
plt.title('Image 1')
plt.axis('off')  # Hide the axes

# Show the second image in the second subplot
plt.subplot(2, 2, 2)
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.title('Image 2')
plt.axis('off')

# Show the matched features in the third subplot
plt.subplot(2, 2, 3)
plt.imshow(matched_image)
plt.title('Matched Keypoints')
plt.axis('off')  # Hide the axes

# Show the warped image in the fourth subplot
plt.subplot(2, 2, 4)
plt.imshow(cv2.cvtColor(img1_warped, cv2.COLOR_BGR2RGB))
plt.title('Panorama')
plt.axis('off')

# Display all images
plt.tight_layout()  # Adjust spacing between subplots
plt.show()

# Save the resulting image
cv2.imwrite('/content/drive/My Drive/ADVIDEO/panorama_result.jpg', img1_warped)


Redo the previous code to include new images in the panoramic view, simulating the mobile panoramic application
